In [ ]:
import csv
from google.cloud import bigquery

In [ ]:
def main():
    # ID do seu projeto
    project_id = "driver-log-414519"

    # Inicializa o cliente BigQuery
    client = bigquery.Client(project=project_id)

    # Defina a query que deseja extrair
    query = """
        SELECT *
        FROM `basedosdados.br_me_caged.microdados_antigos`
        --LIMIT 1000  -- Ajuste conforme necessário, ou remova
    """

    # Configurar o job para rodar em modo BATCH (menos prioritário, pode levar mais tempo para iniciar)
    job_config = bigquery.QueryJobConfig(
        priority=bigquery.QueryPriority.BATCH
    )

    # Executa a query
    query_job = client.query(query, job_config=job_config)

    # Aguarda a conclusão e obtém o iterador de resultados com paginação
    rows_iterable = query_job.result(page_size=50_000)  # Ajuste 'page_size' conforme necessário

    # Nome do arquivo CSV local
    csv_file_name = "microdados_antigos.csv"
    print(f"Extraindo dados diretamente para: {csv_file_name}")

    # Abre o arquivo CSV para escrita
    with open(csv_file_name, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = None
        # Captura o schema para pegar nomes das colunas
        schema = rows_iterable.schema

        # Percorre as páginas retornadas pelo BigQuery
        for page in rows_iterable.pages:
            # Cria o writer e escreve o cabeçalho (header) na primeira página
            if writer is None:
                writer = csv.writer(csv_file)
                header = [field.name for field in schema]
                writer.writerow(header)

            # Escreve cada linha da página atual
            for row in page:
                writer.writerow(row.values())
    print("Extração concluída com sucesso!")

In [ ]:
if __name__ == "__main__":
    main()